# Tutorial 1 Pneumonia Model - ROC Analysis

### Load all necessary libraries, including 'rhino_health'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.axes
import matplotlib.figure
import matplotlib.pyplot as plt
from PIL import Image
import os
import sys
import rhino_health as rh

### Log in to the Rhino Health Platform
Please input the missing values before running the next cell

In [ ]:
print("Logging In")
my_username = #Replace this with your username
my_password = #Replace this with your password. For better security, consider using an environment variable, e.g. - os.getenv("RHP_PASSWORD") 
rhino_api_url = rh.lib.constants.ApiEnvironment.PROD_API_URL
ecr_base_uri = #Paste your ECR name here as a string
session = rh.login(username=my_username, password=my_password, rhino_api_url=rhino_api_url)
print("Logged In")

### Load the results cohort from the project 
Start by getting your results cohort uid from the gui and pasting it below

In [ ]:
site1_results_cohort = # Replace with your cohort UID here as string
cohort = session.cohort.get_cohort(site1_results_cohort)
print(f"Loaded cohort '{cohort.name}'")

### Calculate ROC (underlying results data stays on-prem)

In [ ]:
from rhino_health.lib.metrics import RocAuc

metric_configuration = RocAuc(y_true_variable="Pneumonia",
                              y_pred_variable="Model Score")
results = cohort.get_metric(metric_configuration)

### Plot the ROC with your favorite plotting tool

In [ ]:
import matplotlib.pyplot as plt
import json

roc_metrics = results.output


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
color = colors[0]
linestyle = linestyle_cycle[0]
ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
ax.title.set_text('Overall ROC')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

fig.canvas.draw()
image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
image_to_store.save("Overall_ROC.png", format='png', optimize=True, quality=100)

### Calculate and plot ROC with a Confidence Interval

In [ ]:
import matplotlib.pyplot as plt
import json
from rhino_health.lib.metrics import RocAucWithCI

metric_configuration = RocAucWithCI(
   timeout_seconds = 30.0,
    y_true_variable="Pneumonia",
    y_pred_variable="Model Score",
    confidence_interval=95
)
results = cohort.get_metric(metric_configuration)

In [ ]:
roc_metrics = results.output


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
color = colors[0]
linestyle = linestyle_cycle[0]
tpr_ci = roc_metrics['tpr_ci']
ax.fill_between(roc_metrics['fpr'], tpr_ci[0], tpr_ci[1], alpha=0.33, label='_nolegend_', color=color)
ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
ax.title.set_text('Overall ROC with Confidence Interval')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
fig.canvas.draw()
image_to_store = Image.frombytes('RGB', 
fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
image_to_store.save("Overall_ROC_CI.png", format='png', optimize=True, quality=100)

### Calculate and plot ROC grouped by Gender

In [ ]:
from rhino_health.lib.metrics import RocAuc

metric_configuration = RocAuc(y_true_variable="Pneumonia",
                              y_pred_variable="Model Score",
                             group_by = { 'groupings': ['Gender']})
results = cohort.get_metric(metric_configuration)

In [ ]:
import matplotlib.pyplot as plt
import json

for group in results.output.keys():
    roc_metrics= results.output[group]
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    linestyle_cycle = ['-', '--']
    fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
    color = colors[0]
    linestyle = linestyle_cycle[0]
    ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
    ax.title.set_text(group)
    ax.set_xlabel('1 - Specificity')
    ax.set_ylabel('Sensitivity')
    ax.grid(True)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    fig.canvas.draw()
    image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    image_to_store.save(f"Gender_ROC_{group}.png", format='png', optimize=True, quality=100)


### Plot the ROC by Gender on a single chart

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax =plt.subplots(figsize=[6, 4], dpi=200)


linestyle = linestyle_cycle[0]
for group,color in zip(results.output.keys(),colors):
    roc_metrics= results.output[group]
    ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle,label = group)
    ax.legend(loc='lower right')
ax.title.set_text('ROC by Gender')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

fig.canvas.draw()
image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
image_to_store.save("Gender_ROC.png", format='png', optimize=True, quality=100)


### Upload the visualizations to the Rhino Health Platform

In [ ]:
model_result_uid = # Paste the UID of the model results object for your NVFlare model - look for the (V) icon

In [ ]:
print("Sending visualizations to the Cloud")

import io
import base64
import json

def add_images_to_report(report_data, image_files):
    for image_file in image_files:
        with open(image_file, "rb") as temp_image:
            base_64_image = base64.b64encode(temp_image.read()).decode("utf-8")
            report_data.append(
              {
                 "type": "Image",
                 "data": {
                     "image_filename": image_file,
                     "image_base64": base_64_image,
                 },
                 "width": 100 / len(image_files)
              }
           )

roc_image_files = ('Overall_ROC.png', 'Overall_ROC_CI.png')
gender_image_files = ('Gender_ROC_M.png', 'Gender_ROC_F.png', 'Gender_ROC.png')

report_data = []
report_data.append({"type": "Title", "data": "Overall ROC"})
add_images_to_report(report_data, roc_image_files)
report_data.append({"type": "Title", "data": "ROC by Gender"})
add_images_to_report(report_data, gender_image_files)
    

result = session.post(f"federatedmodelactions/{model_result_uid}/set_report/", 
                      data={"report_data": json.dumps(report_data)})
print('Done')